In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
KAGGLE_HOME_DIR = '/content/drive/MyDrive/kaggle'
os.environ['KAGGLE_HOME_DIR'] = KAGGLE_HOME_DIR

In [ ]:
%cd {KAGGLE_HOME_DIR}

/content/drive/MyDrive/kaggle


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import csv
import time
import torch
import torchvision
from sklearn import preprocessing
from torchvision import transforms, datasets
from tqdm import tqdm
from tensorflow import keras
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,LeakyReLU
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D ,BatchNormalization,Activation
from tensorflow.keras.optimizers import Adam ,RMSprop, SGD, Adagrad, Adamax
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.regularizers import l2
from tensorflow.keras import layers

In [ ]:
SUBMISSION_HEADER = ['Id', 'class']
RANDOM_STATE = 0

In [ ]:
def open_pickled_file(file):
    with open(file, 'rb') as f:
        data = pd.read_pickle(file)
    return np.asarray(data)

In [ ]:
def export_predictions(y_pred):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    filename = KAGGLE_HOME_DIR+'/prediction{}.csv'.format(timestr)
    file = open(filename, 'w+', newline ='')
    with file:    
        writer = csv.writer(file)
        writer.writerow(SUBMISSION_HEADER)
        writer.writerows(enumerate(y_pred))

In [ ]:
def preprocess(X, y):
    X = scale(X)
    y = encode(y)
    return X, y

In [ ]:
def encode(y):
    le = preprocessing.LabelEncoder()
    le.fit(y)
    return le.transform(y)

In [ ]:
def scale(X):
    max_val = np.max(X)
    return X/max_val

In [ ]:
X_train_raw = open_pickled_file(KAGGLE_HOME_DIR+'/x_train.pkl.zip')
y_train_raw = open_pickled_file(KAGGLE_HOME_DIR+'/y_train.pkl')
X_test_raw = open_pickled_file(KAGGLE_HOME_DIR+'/x_test.pkl.zip')

In [ ]:
X_train, y_train = preprocess(X_train_raw, y_train_raw)

In [ ]:
X_test = X_test_raw/255

In [ ]:
Y_enc = to_categorical(y_train, 11)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, Y_enc, test_size=0.2,random_state =0, stratify=Y_enc)

In [ ]:
batch_size = 128
epochs = 5
num_classes = 11

In [ ]:
model = Sequential([
# data_augmentation,
layers.Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(96,96,1),padding='same', kernel_regularizer=l2(0.01),
    bias_regularizer=l2(0.01)),
layers.LeakyReLU(alpha=0.1),
layers.BatchNormalization(),
layers.MaxPooling2D((2, 2),padding='same'),

layers.Conv2D(64, (3, 3), activation='linear',padding='same'),
layers.LeakyReLU(alpha=0.1),
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2),padding='same'),

layers.Conv2D(128, (3, 3), activation='linear',padding='same', kernel_regularizer=l2(0.01),
    bias_regularizer=l2(0.01)),
layers.LeakyReLU(alpha=0.1), 
layers.BatchNormalization(),         
layers.MaxPooling2D(pool_size=(2, 2),padding='same'),

layers.Conv2D(256, (3, 3), activation='linear',padding='same'),
layers.LeakyReLU(alpha=0.1),
layers.BatchNormalization(),                 
layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
layers.Dropout(0.25),

layers.Flatten(),

layers.Dense(128, activation='linear'),
layers.LeakyReLU(alpha=0.1),  
layers.BatchNormalization(),
layers.Dropout(0.25),

layers.Dense(64, activation='linear'),
layers.LeakyReLU(alpha=0.1), 
layers.BatchNormalization(), 
layers.Dropout(0.25),

layers.Dense(num_classes, activation='softmax')])

In [ ]:
model1 = Sequential([
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape = (96,96,1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
    layers.BatchNormalization(),

    # Add convolutional layer consisting of 32 filters and shape of 5x5 with ReLU activation
    # We give strides=2 for space between each sample on the pixel grid
    layers.Conv2D(32, kernel_size = (5,5), strides=2, padding='same', activation='relu'),
    layers.BatchNormalization(),
    # Dropping %40 of neurons
    layers.Dropout(0.4),
    layers.Conv2D(64, kernel_size = (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, kernel_size = (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, kernel_size = (5,5), strides=2, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(128, kernel_size = 4, activation='relu'),
    layers.BatchNormalization(),
    # To be able to merge into fully connected layer we have to flatten
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    # Lets add softmax activated neurons as much as number of classes
    layers.Dense(num_classes, activation = "softmax")
                     
])


In [ ]:
model_master = Sequential([
    layers.Conv2D(96, kernel_size=(3, 3), activation='relu', input_shape=(96,96,1), padding="same"),
    layers.Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same'),
    layers.Conv2D(96, (3, 3), activation='relu', padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3), strides = 2),
    layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'),
    layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'),
    layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'),
    layers.MaxPooling2D(pool_size=(3, 3), strides = 2),
    layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'),
    layers.Conv2D(192, (1, 1), activation='relu'),
    layers.Conv2D(11, (1, 1)),
    layers.GlobalAveragePooling2D(),
    layers.Activation(activation='softmax')
])
    


In [ ]:
def data_augmentation(X, y):
  img = ImageDataGenerator(
      rotation_range = 5,
      horizontal_flip= True,
      zoom_range=0.02,
      # rescale=1./255,
      # shear_range=0.2,
      width_shift_range=0.15, 
      height_shift_range = 0.15)
  Train_batch = img.flow(X, y, batch_size = 32)
  return Train_batch

In [ ]:
# def data_augmentation_valid(X, y):
#   img = ImageDataGenerator(
#       horizontal_flip = True
#   )
#   Train_batch = img.flow(X, y, batch_size = 32)
#   return Train_batch

In [ ]:
dataload = data_augmentation(X_train.reshape(-1,96,96,1),y_train)

In [ ]:
dataloadval = data_augmentation(X_val.reshape(-1,96,96,1), y_val)

In [ ]:
tf.test.is_built_with_cuda()

True

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# callbacks = [EarlyStopping(patience=3, restore_best_weights=True),
#              ReduceLROnPlateau(patience=2),
#              ModelCheckpoint(filepath=KAGGLE_HOME_DIR+'/ImageDataGen_Size256_oneHOT_ClassWeights_Callbacks_SGD_L2.h5', save_best_only=True)]

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, verbose=1)
# lr_scheduler = LearningRateScheduler(scheduler)
checkpoint = ModelCheckpoint(filepath=KAGGLE_HOME_DIR+'/best_model', monitor='val_accuracy', verbose=1, mode= 'max', save_best_only=True, save_weights_only=True)

In [ ]:
%%timeit -n1 -r1 
with tf.device('/GPU:0'):
  model.compile(loss="categorical_crossentropy", optimizer= Adam(learning_rate=0.01), metrics=["accuracy"])
  # This model is overfitting
  # model.fit(X_train, y_train, epochs=50, shuffle= True)
  history = model.fit(dataload , validation_data=dataloadval, epochs=100, shuffle=True, batch_size=batch_size, callbacks=[reduce_lr, checkpoint])
  # Plot history: accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

  # Plot history: loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

Epoch 1/100


In [ ]:
model.load_weights(KAGGLE_HOME_DIR+'/best_model')

In [ ]:
model.evaluate(X_val,y_val)

75/75 [==============================] - 1s 18ms/step - loss: 1.2124 - accuracy: 0.6627


[1.2124394178390503, 0.6627417802810669]

In [ ]:
pred = model.predict(X_test)
pred= np.argmax(pred, axis=1)

In [ ]:
export_predictions(pred)